In [ ]:
# from uuid import getnode as get_mac
# str(hex(get_mac()))[2:]

In [ ]:

from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5 import uic
from PyQt5 import QtWidgets, QtCore
from uuid import getnode as get_mac
import cv2, time, serial, os, copy, numpy as np
# import numpy.core.multiarray as multiarray

MAC = '1a7dda7113'

# ========================== Serial ===============================
class SerialC():
    def __init__(self):
        self.ONLINE = False
        self.flag_init = 0x0100
        self.flag_conf = 0x0200
        self.flag_comm = 0x0300
        self.Vpv = 0
        self.Vhigh = 0
        self.Vlow = 0
        self.Ipv = 0

    def connect(self, port, baudrate, stbar):
        self.port = port
        self.baudrate = baudrate
        try:
            stbar.showMessage('Serial Connecting...')
            self.com = serial.Serial(self.port, baudrate, timeout=1)
            self.ONLINE = True
            time.sleep(1)
            stbar.showMessage('Serial Connected to ' + self.port + ' Port')
            return True
        except :
            stbar.showMessage('Serial Connect Error : Check COM Port')
            return False
        
    def disconnect(self, stbar):
        if self.ONLINE:
            stbar.showMessage('Serial disconnected from ' + self.port + ' Port')
            self.com.close()
            self.ONLINE = False
            
    def send16(self, sig):
        self.com.write(bytes([sig>>8 & 0xFF]))
        self.com.read()
        self.com.write(bytes([sig>>0 & 0xFF]))
        self.com.read()
            
    def read16(self):
        byte_ms = ord(self.com.read())
        byte_ls = ord(self.com.read())
        int16 = byte_ms*256 + byte_ls
        return int16

    def com_init(self):
        self.send16(self.flag_init)
        self.Vpv = self.read16()
        self.Vhigh = self.read16()
        self.Vlow = self.read16()
        self.mode = 'wait'
        return [self.Vpv, self.Vhigh, self.Vlow]

    def com_conf(self, vmin, vmax):
        self.mode = 'config'
        self.send16(self.flag_conf)
        self.send16(vmin)
        self.send16(vmax)
        cb = self.read16()
        self.Vlow = vmin
        self.Vhigh = vmax
        win_main.lbl_vlow.setText('{:.1f}'.format(vmin*0.1))
        win_main.lbl_vhigh.setText('{:.1f}'.format(vmax*0.1))
        self.mode = 'wait'

    def com_comm(self, sig):
        self.mode = 'common'
        self.send16(self.flag_comm | sig)
        self.conf = self.read16()
        self.Vpv = self.read16()
        self.Ipv = self.read16()
        self.mode = 'wait'
        return [self.conf, self.Vpv, self.Ipv]
            
# =================================================================

# ============================ Camera =============================
class Camera():
    def __init__(self):
        self.SENS = 50
        self.ON = False
        self.MARK = [0, [-1, -1, False, 'h'], 
                        [-1, -1, False, 'h'], 
                        [-1, -1, False, 'h'], 
                        [-1, -1, False, 'h'], 
                        [-1, -1, False, 'h'], 
                        [-1, -1, False, 'h']]
        self.MARK_SEL = -1
        self.COLOR = ['r', 'g', 'b', 'h']
        self.REC_ON = False
        self.AUTOREC = False
        self.PTSIZE = 1
        self.SKIP = 1
        
    def initCam(self, width, height, num):
        self.ON = True        
        self.premil = time.time()*1000
        self.CAM = cv2.VideoCapture(num)
        self.width = width
        self.height = height
        self.frame = np.zeros((height, width, 3), dtype=np.uint8)
        time.sleep(0.2)

    def delCam(self):
        self.ON = False
        try: self.REC.release()
        except: pass
        try: 
            self.CAM.release()
            cv2.destroyAllWindows()
        except: pass
        
    def recWrite(self, Vpv, Ipv):
        img = copy.deepcopy(self.frame)
        now = time.gmtime(time.time())
        cv2.putText(img, 'Time: ({:02d}/{:02d}) {:02d}:{:02d}:{:02d}'.format(now.tm_mon, 
                          now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec), \
                          (5, 20), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 1, cv2.LINE_AA)
        if serialc.ONLINE:
            cv2.putText(img, 'Voltage: {:.1f}'.format(Vpv*0.1), \
                        (self.width-130, 20), \
                        cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 1, cv2.LINE_AA)
            cv2.putText(img, 'Current: {:d}'.format(Ipv), \
                        (self.width-130, 40), \
                        cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 1, cv2.LINE_AA)
        self.REC.write(img)

    def startRec(self):
        file = self.getFileName('clip_', '.avi')
        fourcc= cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
        self.REC = cv2.VideoWriter(file, fourcc, 30, (self.width, self.height))
        self.REC_ON = True
        
    def stopRec(self):
        self.REC_ON = False
        self.REC.release()
    
    def getFileName(self, fileHead, exten):
        folder = 'snapshot/'
        os.chdir(folder)
        files = os.listdir()
        for i in range(len(files)):
            file = fileHead + '{:02d}'.format(i) + exten
            if file not in files: break
        os.chdir('../')
        return folder + file
    
    def isBetween(self, a, x, b):
        return x >= a and x <= b
        
    def readPoints(self, img, cb, w, s, Print=False):
        sig = 0b00000000
        if not self.ON:
            return sig
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        for i in range(6): # 6 Point
            if self.MARK[i+1][2] and cb[i].isChecked(): # 찍혀있고 체크되어있으면
                for j in range(4): # 4채널
                    if self.MARK[i+1][3] == self.COLOR[j]: 
                        if self.isBetween(w[i].value(), # 동작점
                                          win_main.buf[i][j],
                                          s[i].value()): # 정지점
                            sig += (1 << i)
        if Print: print('{0:08b}'.format(sig), end='\r')
        return sig
    
    def masking(self, bottom):
        top = self.PTS
        rows, cols, channels = top.shape
        roi = bottom[0:rows, 0:cols]

        img2gray = cv2.cvtColor(top, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)

        img1_fg = cv2.bitwise_and(top, top, mask=mask)
        img2_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

        dst = cv2.add(img1_fg, img2_bg)

        bottom[0:rows, 0:cols] = dst
        return bottom
    
    def capture(self, w, h):
        try: self.CAM
        except: return np.zeros((h, w, 3))
        ret, cvimg = self.CAM.read()
        if not ret:
            cvimg = np.zeros((h, w, 3), dtype=np.uint8)
            win_main.statusBar().showMessage('Camera Capture Error : Check Camera Number')
            self.delCam()
            win_main.btn_cam.setText('Turn On')
            win_main.btn_rec.setText('Recode\nStart')
        cvimg = cv2.flip(cvimg, 1)
        cvimg = cv2.resize(cvimg, (w, h))
        return cvimg
        
    def img2pix(self, cam):
        img = QImage(cam, cam.shape[1], cam.shape[0], cam.shape[1]*3, QImage.Format_RGB888)
        return QPixmap.fromImage(img)
    
    def drawPoints(self, w, h):
        self.PTS = np.zeros((h, w, 3), dtype=np.uint8)
        scale = 1.5
        redius = int(scale*(6 + camera.PTSIZE))
        txtSize = int(scale*(1+camera.PTSIZE*0.15))
        txtOffX = int(-2-4*txtSize)
        txtOffY = int(1+5*txtSize)
        for i in range(6):
            if camera.MARK[i + 1][2]:
                if camera.MARK_SEL == i:
                    cv2.circle(self.PTS, (camera.MARK[i+1][0], camera.MARK[i+1][1]), redius, (150,255,160),-1) # fill
                    cv2.circle(self.PTS, (camera.MARK[i+1][0], camera.MARK[i+1][1]), redius, (100,255,110),1) # outline
                else:
                    cv2.circle(self.PTS, (camera.MARK[i+1][0], camera.MARK[i+1][1]), redius, (150,150,255),-1) # fill
                    cv2.circle(self.PTS, (camera.MARK[i+1][0], camera.MARK[i+1][1]), redius, (200,200,255),1) # outline
                cv2.putText(self.PTS, str(i+1), \
                            (camera.MARK[i+1][0]+txtOffX, camera.MARK[i+1][1]+txtOffY), \
                            cv2.FONT_HERSHEY_PLAIN, txtSize, (255,0,0), 1, cv2.LINE_AA)
                
    def pointing(self, x, y):
        if self.ON:
            self.MARK[(self.MARK[0] % 6) + 1] = [x, y, True, self.MARK[(self.MARK[0] % 6) + 1][3]]
            self.MARK[0] += 1

# =================================================================    
    
serialc = SerialC()
camera = Camera() 

# ========================= Preference ============================
class Preference(QDialog):
    def __init__(self, parent=None):
        super(Preference, self).__init__(parent)
        self.setWindowTitle('Preference')
        uic.loadUi('ui/setting.ui', self)
        
        self.btn_save.clicked.connect(self.btnSave)
        self.btn_cancel.clicked.connect(self.btnCancel)
        self.sp_ptsize.setValue(camera.PTSIZE)
        self.sp_skip.setValue(camera.SKIP)
        
    def btnSave(self):
        camera.PTSIZE = self.sp_ptsize.value()
        camera.SKIP = self.sp_skip.value()
        if serialc.ONLINE:
            while True:
                if serialc.mode == 'wait':
                    serialc.com_conf(win_main.win_prefer.sp_mv.value(), win_main.win_prefer.sp_MV.value())
                    break
        self.close()
        
    def btnCancel(self):
        self.close()
        
# ========================= Thread ================================
class tfThread(QThread):
    threadEvent = QtCore.pyqtSignal(int)
 
    def __init__(self, parent=None):
        super().__init__()
        self.main = parent
 
    def run(self):
        if camera.ON:
            win_main.statusBar().showMessage('Camera Opened')
        while True:
            if camera.ON:
                camera.frame = camera.capture(win_main.frame.geometry().width(), win_main.frame.geometry().height())
                if camera.REC_ON:
                    camera.recWrite(serialc.Vpv, serialc.Ipv)
            else:
                win_main.statusBar().showMessage('Camera Capture Error : Check Camera Number')
                camera.delCam()
                win_main.btn_cam.setText('Turn On')
                win_main.btn_rec.setText('Recode\nStart')
                break
                
        
# =========================== Main ================================
class Main(QMainWindow):
    def __init__(self, parent=None):
        super(Main, self).__init__(parent)
        self.setWindowTitle('cam_exam')
        uic.loadUi('ui/main.ui', self)
        self.FPS = 30
        self.initUI()
        self.th = tfThread(self)
        self.buf = np.zeros((6, 4), np.int32)
        self.fpsbuf = np.zeros((10,))
        self.bufidx=0
        self.premil = time.time()*1000
        
    def initUI(self):
        self.act_prefer.triggered.connect(self.openPrefer)
        self.btn_cam.clicked.connect(self.camSwitch)
        self.sld_fps.setValue(self.FPS)
        self.sld_fps.valueChanged.connect(self.setFps)
        self.sld_sens.setValue(camera.SENS)
        self.sld_sens.valueChanged.connect(self.setSens)
        self.btn_serial.clicked.connect(self.serialSwitch)
        self.btn_snap.clicked.connect(self.snapshot)
        self.btn_rec.clicked.connect(self.recode)
        self.btn_autorec.clicked.connect(self.btnAutoRec)
        self.le_baud.setText('9600')
        self.lbl_fps.setText('  FPS : ' + str(self.FPS))
        self.lbl_sens.setText('Sens : ' + str(camera.SENS))
        self.statusBar()
        self.timer = QTimer()
        self.timer.timeout.connect(self.MainLoopFunc)
        self.lbl_r = [self.lbl_r1, self.lbl_r2, self.lbl_r3, self.lbl_r4, self.lbl_r5, self.lbl_r6]
        self.lbl_g = [self.lbl_g1, self.lbl_g2, self.lbl_g3, self.lbl_g4, self.lbl_g5, self.lbl_g6]
        self.lbl_b = [self.lbl_b1, self.lbl_b2, self.lbl_b3, self.lbl_b4, self.lbl_b5, self.lbl_b6]
        self.lbl_h = [self.lbl_h1, self.lbl_h2, self.lbl_h3, self.lbl_h4, self.lbl_h5, self.lbl_h6]
        self.lbl_c = [self.lbl_c1, self.lbl_c2, self.lbl_c3, self.lbl_c4, self.lbl_c5, self.lbl_c6]
        self.cb = [self.cb_1, self.cb_2, self.cb_3, self.cb_4, self.cb_5, self.cb_6]
        self.sp_w = [self.sp_w1, self.sp_w2, self.sp_w3, self.sp_w4, self.sp_w5, self.sp_w6]
        self.sp_s = [self.sp_s1, self.sp_s2, self.sp_s3, self.sp_s4, self.sp_s5, self.sp_s6]
        self.color_white, self.color_red = ['rgb(255, 255, 255);', 'rgb(255, 150, 150);']
        self.color_nor = self.color_white
        self.color_sel = self.color_red
        for i in range(6):
            self.lbl_r[i].setStyleSheet('background-color: ' + self.color_nor)
            self.lbl_g[i].setStyleSheet('background-color: ' + self.color_nor)
            self.lbl_b[i].setStyleSheet('background-color: ' + self.color_nor)
            self.lbl_h[i].setStyleSheet('background-color: ' + self.color_sel)
        self.show()
        self.loadLogo('ui/logo.png')
        self.win_prefer = Preference(self)
        self.timeout = 30
        
    def checkMAC(self, add):
        if(str(hex(get_mac()))[2:] != add):
            self.statusBar().showMessage('MAC address Error')
            time.sleep(2)
            camera.delCam()
            self.close()
        
    def openPrefer(self):
        self.win_prefer.show()
        self.win_prefer.sp_mv.setValue(serialc.Vlow)
        self.win_prefer.sp_MV.setValue(serialc.Vhigh) 
        
    def isOnObject(self, x, y, obj, offset):
        x, y = [x - offset.geometry().left(), y - offset.geometry().top()]
        if x < obj.geometry().left() or y < obj.geometry().top() or \
            x > obj.geometry().right() or y > obj.geometry().bottom():
            return False
        else : return True
    
    def keyPressEvent(self, e):
        if e.key() == Qt.Key_Escape: self.close()
        
    def distance(self, x, y, mark):
        return np.sqrt((x - mark[0])*(x - mark[0]) + (y - mark[1])*(y - mark[1]))
        
    def mouseReleaseEvent(self, e):
        camera.MARK_SEL = -1
    
    def mouseMoveEvent(self, e):
        x = e.x()-self.frame.geometry().left()-self.FrameBox.geometry().left()
        y = e.y()-20-self.frame.geometry().top()-self.FrameBox.geometry().top()
        if camera.MARK_SEL != -1:
            camera.MARK[camera.MARK_SEL+1][0] = x - self.markDif[0]
            camera.MARK[camera.MARK_SEL+1][1] = y - self.markDif[1]
        
    def mousePressEvent(self, e):
        x = e.x()-self.frame.geometry().left()-self.FrameBox.geometry().left()
        y = e.y()-20-self.frame.geometry().top()-self.FrameBox.geometry().top()
        if self.isOnObject(x, y, self.frame, self.FrameBox):
            for i in np.arange(5, -1, -1):
                if self.distance(x, y, camera.MARK[i+1]) < np.sqrt(camera.PTSIZE)*10:
                    camera.MARK_SEL = i
                    self.markDif = [x - camera.MARK[i+1][0], y - camera.MARK[i+1][1]]
                    break
            if camera.MARK_SEL == -1:
                camera.pointing(x, y)
        for i in range(6):
            if self.isOnObject(x, y, self.lbl_r[i], self.GridBox):
                camera.MARK[i+1][3] = 'r'
                self.lbl_r[i].setStyleSheet('background-color: ' + self.color_sel)
                self.lbl_g[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_b[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_h[i].setStyleSheet('background-color: ' + self.color_nor)
            elif self.isOnObject(x, y, self.lbl_g[i], self.GridBox):
                camera.MARK[i+1][3] = 'g'
                self.lbl_r[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_g[i].setStyleSheet('background-color: ' + self.color_sel)
                self.lbl_b[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_h[i].setStyleSheet('background-color: ' + self.color_nor)
            elif self.isOnObject(x, y, self.lbl_b[i], self.GridBox):
                camera.MARK[i+1][3] = 'b'
                self.lbl_r[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_g[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_b[i].setStyleSheet('background-color: ' + self.color_sel)
                self.lbl_h[i].setStyleSheet('background-color: ' + self.color_nor)
            elif self.isOnObject(x, y, self.lbl_h[i], self.GridBox):
                camera.MARK[i+1][3] = 'h'
                self.lbl_r[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_g[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_b[i].setStyleSheet('background-color: ' + self.color_nor)
                self.lbl_h[i].setStyleSheet('background-color: ' + self.color_sel) 
        self.update()
            
    def readMark(self, cam):
        if not cam.ON: return
        img = cv2.resize(camera.frame, (self.frame.geometry().width(), self.frame.geometry().height()))
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        self.readMark_getValue(cam.MARK, img, gray)
    
    def readMark_getValue(self, mark, img, gray):
        b, g, r = cv2.split(img)
        for i in range(6):
            if mark[i+1][2] and self.cb[i].isChecked():
                self.lbl_b[i].setText(str(self.readMark_average(i, 0, mark, b, i, camera.PTSIZE)))
                self.lbl_g[i].setText(str(self.readMark_average(i, 1, mark, g, i, camera.PTSIZE)))
                self.lbl_r[i].setText(str(self.readMark_average(i, 2, mark, r, i, camera.PTSIZE)))
                self.lbl_h[i].setText(str(self.readMark_average(i ,3, mark, gray, i, camera.PTSIZE)))
                
    def probablity(self, f):
        return np.random.randint(0, 100) < f*100
                
    def readMark_average(self, i, j, mark, img, k, size):
        size = size*2 + 1
        num = size*size
        size = int((size - 1) * 0.5)
        roi = int(np.sum(img[mark[k+1][1]-size:mark[k+1][1]+size+1, mark[k+1][0]-size:mark[k+1][0]+size+1])/num)
        if self.probablity(abs(roi - self.buf[i][j])*0.1): # 차이*0.1의 확률로 값이 변함
            self.buf[i][j] = roi
        return self.buf[i][j]
                
    def paintEvent(self, e):
        camera.drawPoints(self.frame.geometry().width(), self.frame.geometry().height())
        
    def setFps(self):
        self.timeout = self.sld_fps.value()
        self.lbl_fps.setText('  Timeout : ' + str(self.timeout))
        if camera.ON:
            self.timer.stop()
            self.timer.start(1000. / self.timeout)
        
    def setSens(self):
        camera.SENS = self.sld_sens.value()
        self.lbl_sens.setText('Sens : ' + str(camera.SENS))
        
    def camSwitch(self):
#         self.checkMAC(MAC)
        if not camera.ON:
            self.btn_cam.setText('Turn Off')
            self.statusBar().showMessage('Camera Opening...')
            camera.initCam(self.frame.geometry().width(), self.frame.geometry().height(), self.sp_cam.value())
            self.th.start()
            self.frame.setFixedSize(self.frame.size())
            self.img_o = camera.capture(self.frame.geometry().width(), self.frame.geometry().height())
            cv2.imwrite('snapshot/img_o.jpg', self.img_o)
            self.timer.start(1000. / self.timeout)
            self.camIdx = 0
        else:
            self.btn_cam.setText('Turn On')
            try: self.th.terminate()
            except: pass
            self.frame.setMaximumSize(QSize(16777215, 16777215))
            self.frame.setMinimumSize(QSize(0, 0))
            self.btn_rec.setStyleSheet('')
            camera.delCam()
            self.frame.clear()
            self.timer.stop()
#             if serialc.ONLINE: self.serialSwitch()
            self.btn_rec.setText('Recode\nStart')
            self.showLogo()
#             try: camera.REC.release()
#             except: pass
            
    def serialSwitch(self):
        if serialc.ONLINE:
            serialc.disconnect(self.statusBar())
            self.btn_serial.setText('Start')
        elif serialc.connect('COM' + str(self.sp_port.value()), int(self.le_baud.text()), self.statusBar()):
            self.btn_serial.setText('Stop')
            self.timer.start(1000. / self.timeout)
            self.initializeCommunicate()
            
    def snapshot(self):
        if not camera.ON: self.statusBar().showMessage('Turn on camera first'); return
        file = camera.getFileName('snapshot_', '.jpg')
        cv2.imwrite(file, camera.frame)
        self.statusBar().showMessage(file)
        
    def recode(self):
        if not camera.ON: self.statusBar().showMessage('Turn on camera first'); return
        if not camera.REC_ON:
            camera.startRec()
            self.statusBar().showMessage('Recoding Started')
            self.btn_rec.setText('Recode\nStop')
            self.btn_rec.setStyleSheet('background-color: ' + self.color_sel)
        else:
            camera.stopRec()
            self.statusBar().showMessage('Recoding Stopped')
            self.btn_rec.setText('Recode\nStart')
            self.btn_rec.setStyleSheet('')
        
    def calFPS(self):
        millis = time.time()*1000
        self.fpsbuf[self.bufidx] = 1000/(millis - self.premil+1)
        self.bufidx = (self.bufidx + 1)%len(self.fpsbuf)
        self.FPS = int(np.mean(self.fpsbuf))
        self.premil = millis
        
    def MainLoopFunc(self): # ================================================================================
        if camera.ON:
            self.img_p = cv2.cvtColor(camera.frame, cv2.COLOR_BGR2RGB)
            if self.camIdx % camera.SKIP == 0:
                self.compare(self.img_o, self.img_p)
                self.readMark(camera)
                self.cam_bit6 = camera.readPoints(self.img_p, self.cb, self.sp_w, self.sp_s, Print=False)
                self.channelRamp(self.cam_bit6)
                self.calFPS()
                self.prt.setText('\tFPS : {:02d}'.format(self.FPS))
            self.camIdx += 1
            img_pts = camera.masking(self.img_p)
            self.frame.setPixmap(camera.img2pix(img_pts))
        self.SerialCommunicate()
        
    def channelRamp(self, sig):
        for i in range(6):
            if (sig>>i) & 0x01:
                self.lbl_c[i].setStyleSheet('background-color: ' + self.color_sel)
            else :
                self.lbl_c[i].setStyleSheet('background-color: ' + self.color_nor)
        
    def initializeCommunicate(self):
        if not serialc.ONLINE :
            return
        vp, vh, vl = serialc.com_init()
        
        self.lbl_vpv.setText('{:.1f}'.format(vp*0.1))
        self.lbl_vhigh.setText('{:.1f}'.format(vh*0.1))
        self.lbl_vlow.setText('{:.1f}'.format(vl*0.1))
        
    def SerialCommunicate(self):
        if not serialc.ONLINE or serialc.mode != 'wait': return
        if camera.ON: bit=self.cam_bit6
        else: bit=0x00 
        st, vp, ip = serialc.com_comm(bit)
        self.setVoltLabelColor(st)
        
        self.lbl_vpv.setText('{:.1f}'.format(vp*0.1))
        self.lbl_apv.setText(str(ip))
        
    def setVoltLabelColor(self, status):
        if status & 0x01 : # too low voltage
            self.lbl_vlow.setStyleSheet('background-color: ' + self.color_red)
        else:
            self.lbl_vlow.setStyleSheet('background-color: ' + self.color_white)
        if (status >> 1) & 0x01: # too high voltage
            self.lbl_vhigh.setStyleSheet('background-color: ' + self.color_red)
        else:
            self.lbl_vhigh.setStyleSheet('background-color: ' + self.color_white)
        
    def sens(self, img_o, img_p):
        rtn = img_p - img_o
        return int(3*np.sum(np.multiply(rtn, rtn))/(img_o.shape[0]*img_o.shape[1]))

    def compare(self, img_o, img_p):
        if self.sens(img_o, img_p)//(camera.SENS*2) > 0:
            if camera.AUTOREC:
                self.autoRecode()
            t = time.localtime()
            millis = int(round(time.time()*10)%10)
            self.lbl_autorec.setText('Motion Detect\n[{:02d}:{:02d}:{:02d}.{:01d}]'.format(t.tm_hour, 
                                                                        t.tm_min, t.tm_sec, millis))
            self.img_o = self.img_p.copy()
            
    def autoRecode(self):
        if not camera.REC_ON:
            camera.startRec()
            self.statusBar().showMessage('Recoding Started')
            self.btn_rec.setText('Recode\nStop')
            self.btn_rec.setStyleSheet('background-color: ' + self.color_sel)
            
    def btnAutoRec(self):
        if not camera.AUTOREC:
            camera.AUTOREC = True
            self.btn_autorec.setText('Auto Rec.\nOff')
        else:
            camera.AUTOREC = False
            self.btn_autorec.setText('Auto Rec.\nOn')
    
    def showLogo(self):
        return
        self.frame.setPixmap(self.pixmap)
    
    def loadLogo(self, file):
        logo = cv2.imread(file)
        logo_h, logo_w, _ = logo.shape
        pad = [int(1.0*logo_h), int(2*logo_w)]
        logo = cv2.copyMakeBorder(logo, pad[0], pad[0], pad[1], pad[1],
                                  cv2.BORDER_CONSTANT, value=(255, 255, 255))
        logo_h, logo_w, _ = logo.shape
        qimg = QImage(bytearray(logo.data), logo_w, logo_h,
                      logo_w*3, QImage.Format_RGB888).rgbSwapped()
        self.frame_w, self.frame_h = [self.frame.geometry().right() - self.frame.geometry().left(), 
                                      self.frame.geometry().bottom() - self.frame.geometry().top()]
        self.pixmap = QPixmap.fromImage(qimg).scaled(self.frame_w, self.frame_h)
        self.showLogo()
        
    def closeEvent(self, event):
        camera.delCam()
        serialc.disconnect(self.statusBar())
        self.timer.stop()
        
app = QApplication([])
app.aboutToQuit.connect(app.deleteLater)
win_main = Main()
print(app.exec_(),'\b\b')
